# AXI Stream 

In [95]:
from pynq import Overlay
ol_adds = Overlay("overlay_4/axis_adder.bit")


In [96]:

from time import time
num_data = 1000

In [97]:
import numpy as np
from pynq import allocate

# xlnk = Xlnk()
in_buffer = allocate(shape=(num_data,), dtype=np.uint32, cacheable=1)
out_buffer0 = allocate(shape=(num_data,), dtype=np.uint32, cacheable=1)
out_buffer1 = allocate(shape=(num_data,), dtype=np.uint32, cacheable=1)

In [98]:
for i in range(num_data):
    in_buffer[i] = (np.random.randint(0,15)<<4)+np.random.randint(0,15)

In [99]:
for i in range(10):
    print(hex(in_buffer[i]))

0x2e
0x1
0xce
0x81
0x86
0xa
0xe5
0x69
0xc2
0x28


In [100]:
for i in range(10):
    print('0x' + format(out_buffer0[i], '02x'))
for i in range(10):
    print('0x' + format(out_buffer1[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [101]:
import pynq.lib.dma
dma_ip = ol_adds.axi_dma_0

In [102]:
def adds_data():
    dma_ip.sendchannel.transfer(in_buffer)
    dma_ip.recvchannel.transfer(out_buffer1)
    dma_ip.sendchannel.wait()
    dma_ip.recvchannel.wait()

In [103]:
t1 = time()
adds_data()
t2 = time()
t_adds = t2-t1
print(t_adds)

0.006020545959472656


In [69]:
ol_adds?

In [104]:
for i in range(10):
    print('0x' + format(out_buffer1[i], '02x'))

0x10
0x01
0x1a
0x09
0x0e
0x0a
0x13
0x0f
0x0e
0x0a


# AXI Lite

In [105]:
from pynq import Overlay

ol_add = Overlay('overlay_4/axil_adder.bit')


adder_ip = ol_add.axi_adder_0

In [106]:
def add_data():
    for i in range(len(in_buffer)):
        adder_ip.write(0x0, int(in_buffer[i]>>4))
        adder_ip.write(0x4, int(in_buffer[i]))
        out_buffer0[i] = adder_ip.read(0x8)

In [107]:
t1 = time()
add_data()
t2 = time()
t_add = t2-t1

# Performance

In [109]:
for i in range(num_data):
    assert out_buffer0[i] == out_buffer1[i], \
        f'unmatched data: {out_buffer0[i]}!={out_buffer1[i]} '\
        f'for {i}-th input {in_buffer[i]}.'

print('time used for axi lite adder: {}'.format(t_add))
print('time used for axi stream adder: {}'.format(t_adds))
#print('improvement')

time used for axi lite adder: 0.15281176567077637
time used for axi stream adder: 0.006020545959472656
